# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [100]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

[program['id'] for program in data['programs']]

['circuit-runner', 'estimator', 'sampler', 'qasm3-runner']

## (optional) get temporary Access token from Auth API via API Token

This is especially useful if you would like control over tokens, such as token invalidation. Alternatively, you can work directly with your IBM Quantum Platform API token.

In [101]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [102]:
url_backends = 'https://runtime-us-east.quantum-computing.ibm.com/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['ibmq_qasm_simulator', 'ibmq_qasm_simulator', 'simulator_extended_stabilizer', 'simulator_mps', 'simulator_stabilizer'] ...


## Run a job

### set up qasm circuit

In [103]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

## transpile circuit via transpilation service API

In [104]:
backend='ibm_peekskill'

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
        }
body= {
    "qasm_circuits": qasm_string,
}
params = {
        "backend": backend,
        "optimization_level": 3,
        "use_ai": False,
        }

resp = requests.post(
            "https://cloud-transpiler.quantum.ibm.com/transpile",
            headers=headers,
            json=body,
            params=params,
        )


In [105]:
task_id=resp.json()['task_id']
task_id

'658d9dfb-f2bc-41b6-b0dc-b46a1a825b5e'

### Request the results using the task_id

In [106]:
res = requests.get(url=f"https://cloud-transpiler.quantum.ibm.com/transpile/{task_id}", headers=headers)
print(res.json())

{'state': 'SUCCESS', 'result': [{'qasm': 'OPENQASM 3.0; include "stdgates.inc"; bit[1] c; x $0; c[0] = measure $0;', 'success': True, 'layout': {'initial': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 'final': [0]}}]}


In [107]:
if res.json().get("state") == "SUCCESS":
    resulting_qasm=res.json().get("result")[0].get("qasm")

print(resulting_qasm)

OPENQASM 3.0; include "stdgates.inc"; bit[1] c; x $0; c[0] = measure $0;


### run circuit via API

In [108]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "start_session": "False", #set to False if you just need to run a single job.  
    "params": {
        "circuits": [resulting_qasm]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cqt1zh688ev000811h70","backend":"ibm_peekskill"}


### Get job result

In [114]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': -0.015310037859408642, '1': 1.0153100378594087}],
 'metadata': [{'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.733130625518517,
   'readout_mitigation_time': 0.020670877769589424}]}

## (optional) Create Session 

c.f. documentation at https://docs.quantum.ibm.com/api/runtime

In [115]:
import json
 
sessionsUrl = "https://api.quantum-computing.ibm.com/runtime/sessions"
 
headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
  "Content-Type": "application/json" 
}
 
payload = json.dumps({
  "backend": "ibm_peekskill",
  "instance": "ibm-q-internal/dev-sys-software/internal-test"
})
 
response = requests.request("POST", sessionsUrl, data=payload,  headers=headersList)
 
sessionId = response.json()['id']
 
print(response.json())

{'id': 'cqt1zyzch2mg008qtzkg'}


### Submit job(s) against resulting Session ID

In [116]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "session_id": sessionId, # This specifies the previously created Session 
    "params": {
        "circuits": [resulting_qasm]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cqt1zzz9nfw0008hrfd0","backend":"ibm_peekskill","session_id":"cqt1zyzch2mg008qtzkg"}


### Get job result

In [129]:
response_status= requests.get(url+'/'+job_id, headers=headers)
response_status.json().get('state')

{'status': 'Completed', 'reason': '', 'reasonCode': None}

In [130]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': 0.010314490848417368, '1': 0.9896855091515826}],
 'metadata': [{'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.6714627828497903,
   'readout_mitigation_time': 0.0071028415113687515}]}

### close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users.

In [131]:
closureURL="https://api.quantum-computing.ibm.com/runtime/sessions/"+sessionId+"/close"

headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
}
 
payload = json.dumps({
  "backend": "ibm_peekskill",
  "instance": "ibm-q-internal/dev-sys-software/internal-test"
})
 
closure_response = requests.request(
    "DELETE",
    closureURL, 
    headers=headersList)

print("Session closure response ok?:",closure_response.ok,closure_response.text)

Session closure response ok?: True 


## (Optional) Invalidate Token 

In [93]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## (Optional) Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [94]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)

if logout_response.status_code == 200:
    job_id = logout_response.json().get('id')
    print("Job created:",logout_response.text)
elif logout_response.status_code == 401:
    print("invalid credentials. Access token should be successfully invalidated.")
else:
    print(logout_response.text,"\n")
    print(f"Error: {logout_response.status_code}")

invalid credentials. Access token should be successfully invalidated.
